In [1]:
import torch
from torch import nn
import torchviz

import vit
import scipy.io
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from importlib import reload

/home/amintehrani/.pyenv/versions/3.12.8/envs/torchgeo/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from mat import mat

(1, 703)

In [ ]:
all_images = np.transpose(mat['all_images'], (0, 2, 3, 1))
train_indices = mat['train_loc'].squeeze()
val_seen_indices = mat['val_seen_loc'].squeeze()
val_unseen_indices = mat['val_unseen_loc'].squeeze()
test_seen_indices = mat['test_seen_loc'].squeeze()
test_unseen_indices = mat['test_unseen_loc'].squeeze()

total_images = all_images.shape[0]
print(train_indices.shape[0], val_seen_indices.shape[0], val_unseen_indices.shape[0], test_seen_indices.shape[0], test_unseen_indices.shape[0])



13039 3234 3721 4990 7440
32424 32424
